# _Influencia de la duración de piststops en carreras de F1_

* Hipótesis:  

- Mejor media de tiempos por carrera y temporada aumenta las posiciones en carrera y al final de temporada  

- Idem para constructores  

- Diferencias entre conductores de una mismo equipo

<img src="F1_logo.jpg"  width="200"> <p>
<img src="pitStopAstonMartin.jpg"  width="600" height="300">

### <span style="color:RED"> DATASET</span>

#### CSV's EXTRAIDOS DE ergast.com  

* Ofrece resultados de todas las carreras desde 1950 hasta hoy.  

* Paquete ergast_py  
https://pypi.org/project/ergast-py/

* API con datos actualizados tras cada carrera  
  
  * API documentación http://ergast.com/mrd/  
    
* Libreria propia para acceder a la API  
    pyERGAST


#### Ofrecen todo el paquete de csv hasta la carrera más reciente  
* f1db_csv.zip  
http://ergast.com/downloads/f1db_csv.zip  

* Diagrama de la BD  
http://ergast.com/images/ergast_db.png  

* Esquema de la BD  
http://ergast.com/docs/f1db_user_guide.txt  

Cada csv tiene una sola base de datos en UTF-8  
La primera línea son las cabeceras de las columnas. 




In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import re

* 13 csv

In [2]:
circuits=pd.read_csv(r'./circuits.csv')
constructor_results=pd.read_csv(r'./constructor_results.csv')
constructor_standings=pd.read_csv(r'./constructor_standings.csv')
driver_standings=pd.read_csv(r'./driver_standings.csv')
drivers=pd.read_csv (r'./drivers.csv')
lap_times=pd.read_csv(r'./lap_times.csv')
pit_stops=pd.read_csv(r'./pit_stops.csv')
qualifying=pd.read_csv(r'./qualifying.csv')
races=pd.read_csv(r'./races.csv')
results=pd.read_csv(r'./results.csv')
seasons=pd.read_csv(r'./seasons.csv')
sprint_results=pd.read_csv(r'./sprint_results.csv')
status=pd.read_csv(r'./status.csv')


In [4]:
status[status.status.str.contains("[Tt]yre")]
# usar regex para filtrar valores de columnas

,statusId,status
26,27,Tyre
58,59,Tyre puncture


In [9]:
status.query('status.str.contains("[Tt]yre")')

,statusId,status
26,27,Tyre
58,59,Tyre puncture


* Para el primer proyecto EDA, los más interesantes son:  

    * resultados                    | *results.csv*  
    * conductores                   |  *drivers.csv*  
    * carreras                      |  *races.csv*  
    * resultados de constructores por campeonato   | *constructor_standings*  
    * resultados de constructores por carrera   | *constructor_results*  
    * pit stops                     | *pit_stops*  
    * status (finalizado/otros)     | *status*


In [78]:
races11_23=races[["year", "raceId"]][races.raceId>841]
# seleccionar carreras desde el 2011 en adelante, cuando cambió la norma y por seguridad ya no se permite repostaje

In [58]:
# merge resultados con races09_23

results09_23 = pd.merge(races09_23,results, on ='raceId')

In [49]:
results09_23.head(2)

,year,raceId,resultId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,2010,337,20323,4,6,8,3,1,1,1,25.0,49,1:39:20.396,5960396,45,1,1:58.287,191.706,1
1,2010,337,20324,13,6,7,2,2,2,2,18.0,49,+16.099,5976495,38,5,1:59.732,189.392,1


* DataFrame: statusId1 
* df filtrar results09_23 por statusId=1. *Conductores que han terminado la carrera*

In [59]:
# statusId1 
# df filtrar results09_23 por statusId=1. *Conductores que han terminado la carrera*

statusId1 = results09_23[results09_23.statusId==1]
statusId1.sample(2)

,year,raceId,resultId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
4914,2021,1063,25203,8,51,7,0,18,18,18,0.0,1,+36.054,243125,\N,0,\N,\N,1
2843,2016,959,23165,807,10,27,8,7,7,7,6.0,67,+1:10.049,5514249,46,12,1:20.056,205.686,1


In [66]:
status[status.statusId==11]

,statusId,status
10,11,+1 Lap
